In [1]:
!nvidia-smi

Tue Feb 15 10:37:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers
!pip install simpletransformers
!pip install wandb

  Using cached transformers-4.16.2-py3-none-any.whl (3.5 MB)
  Using cached huggingface_hub-0.4.0-py3-none-any.whl (67 kB)
  Using cached PyYAML-6.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (596 kB)
     |████████████████████████████████| 6.8 MB 4.0 MB/s 
  Using cached sacremoses-0.0.47-py2.py3-none-any.whl (895 kB)
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Using cached simpletransformers-0.63.4-py3-none-any.whl (248 kB)
     |████████████████████████████████| 9.7 MB 4.2 MB/s 
     |████████████████████████████████| 1.2 MB 47.2 MB/s 
  Using cached wandb-0.12.10-py2.py3-none-any.whl (1.7 MB)
  Using cached seqeval-1.2.2.tar.gz (43 kB)
     |████████████████████████████████| 311 kB 46.4 MB/s 
  Using cached shortuuid-1.0.8-py3-none-any.whl (9.5 kB)
  Using cached sentry_sdk-1.5.5-py2.py3-none-any.whl (144 kB)
  Using cached

In [3]:
import pandas as pd
import numpy as np
from simpletransformers.classification import ClassificationModel
from sklearn.preprocessing import LabelEncoder

In [4]:
data = pd.read_table('/content/B3DB_classification.tsv')
X = data['SMILES']
target=data['BBB+/BBB-'].values.tolist()
label_encoder = LabelEncoder()
y_classification = label_encoder.fit_transform(target)

In [5]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, y_classification, test_size = 0.2, random_state = 42)
xtrain, xvalid, ytrain, yvalid = train_test_split(xtrain, ytrain, test_size = 0.16, random_state = 42)

In [6]:
Xtrain = {'text':xtrain, 'labels':ytrain}
Xvalid = {'text':xvalid, 'labels':yvalid}
Xtest = {'text':xtest, 'labels':ytest}
train_df = pd.DataFrame(Xtrain)
valid_df = pd.DataFrame(Xvalid)
test_df = pd.DataFrame(Xtest)

In [7]:
model = ClassificationModel('roberta', 'DeepChem/ChemBERTa-77M-MLM', args = {
'evaluate_during_training' : True,
'evaluate_during_training_verbose' : True,
'evaluate_during_training_steps' : 82,
'use_cached_eval_features ':True,
'use_early_stopping': True,
'early_stopping_delta': 0.01,
'early_stopping_metric':"auroc",
'early_stopping_metric_minimize' : False,
'early_stopping_patience' : 15,
'max_seq_length' : 128,
'num_train_epochs' : 20,
'train_batch_size' : 32,
'learning_rate' : 4e-5,
'reprocess_input_data': True, 
'overwrite_output_dir': True,
'auto_weights': True
}, num_labels=2,) # You can set class weights by using the optional weight argument


Downloading:   0%|          | 0.00/631 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepChem/ChemBERTa-77M-MLM were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj

Downloading:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/6.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420 [00:00<?, ?B/s]

In [8]:
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


In [9]:
out = model.train_model(train_df, eval_df=valid_df,args={'wandb_project': 'bbb_model'})

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/5245 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_2_2
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for training.


...

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_2
INFO:simpletransformers.classification.classification_model:{'mcc': 0.7537920478305444, 'tp': 307, 'tn': 579, 'fp': 58, 'fn': 56, 'auroc': 0.9497018133381769, 'auprc': 0.916522801471199, 'eval_loss': 0.2987495889663696}
INFO:simpletransformers.classification.classification_model: No improvement in auroc
INFO:simpletransformers.classification.classification_model: Current step: 15
INFO:simpletransformers.classification.classification_model: Early stopping patience: 15
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_2
INFO:simpletransformers.classification.classification_model:{'mcc': 0.7537920478305444, 'tp': 307, 'tn': 579, 'fp': 58, 'fn': 56, 'auroc': 0.9497299237558978, 'auprc': 0.916910732280318, 'eval_loss': 0.30177796459198}
INFO:simpletransformers.classification.classification_model: Patience of 15 steps reached
INFO:simpletransformers.classification.classification_model: Training terminated.
INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.


In [10]:

model = ClassificationModel(
    "roberta", "/content/outputs/best_model",args={'wandb_project': 'bbb_model'}
)

In [11]:
import sklearn
result, model_outputs, wrong_predictions = model.eval_model(test_df,acc=sklearn.metrics.accuracy_score)
print(result)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1562 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_2


Running Evaluation:   0%|          | 0/196 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for evaluation.


Training loss,███▅▄▅▃▅▃▆▄▆▂▃▂▂▃▂▃▂▂▃▃▃▃▂▂▂▃▂▁▃▂▃▁▂▁▂▂▃
auprc,▁▆▆▇▇▇▇▇▇▇▇█████████████████████████████
auroc,▁▆▆▇▇▇▇▇▇▇▇█████████████████████████████
eval_loss,█▆▆▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
fn,█▄▄▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
fp,▁▂▂█▇▆▇▇▇▇▇▅▇▇▇▆▇▇██▆▆▆▇▅▇▇▅▆▅▆▆▆▆▆▆▆▆▆▆
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
lr,▂▄▆███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
mcc,▁▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████████████
tn,█▇▇▁▂▃▂▂▂▂▂▄▂▂▂▃▂▂▁▁▃▃▃▂▄▂▂▄▃▄▃▃▃▃▃▃▃▃▃▃
tp,▁▅▅▇▇▇▇▇▇▇▇▇███▇████████████████████████


wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.
INFO:simpletransformers.classification.classification_model:{'mcc': 0.6641610812086844, 'tp': 447, 'tn': 872, 'fp': 129, 'fn': 114, 'auroc': 0.9228917250307624, 'auprc': 0.8769095024836031, 'acc': 0.8444302176696543, 'eval_loss': 0.3615753127604115}


{'mcc': 0.6641610812086844, 'tp': 447, 'tn': 872, 'fp': 129, 'fn': 114, 'auroc': 0.9228917250307624, 'auprc': 0.8769095024836031, 'acc': 0.8444302176696543, 'eval_loss': 0.3615753127604115}


In [12]:
result, model_outputs, wrong_predictions = model.eval_model(valid_df,acc=sklearn.metrics.accuracy_score)
print(result)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_2


Running Evaluation:   0%|          | 0/125 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for evaluation.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.717611850367763, 'tp': 300, 'tn': 569, 'fp': 68, 'fn': 63, 'auroc': 0.9448841202087956, 'auprc': 0.9105836691861008, 'acc': 0.869, 'eval_loss': 0.3036353607177734}


{'mcc': 0.717611850367763, 'tp': 300, 'tn': 569, 'fp': 68, 'fn': 63, 'auroc': 0.9448841202087956, 'auprc': 0.9105836691861008, 'acc': 0.869, 'eval_loss': 0.3036353607177734}


In [13]:
!sudo apt-get install git-lfs
!git lfs install
!git config --global credential.helper store

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-compute-11-1
  cuda-nsight-systems-10-1 cuda-nsight-systems-

In [14]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/token.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token


In [15]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("/content/outputs/best_model")
model.push_to_hub("Parsa/BBB_prediction_classification")

tokenizer = AutoTokenizer.from_pretrained("/content/outputs/best_model")
tokenizer.push_to_hub("Parsa/BBB_prediction_classification")

Cloning https://huggingface.co/Parsa/BBB_prediction_classification into local empty directory.


Upload file pytorch_model.bin:   0%|          | 3.38k/13.1M [00:00<?, ?B/s]

To https://huggingface.co/Parsa/BBB_prediction_classification
   77df0f6..6bd97f8  main -> main

   77df0f6..6bd97f8  main -> main

To https://huggingface.co/Parsa/BBB_prediction_classification
   6bd97f8..1f25e11  main -> main

   6bd97f8..1f25e11  main -> main



'https://huggingface.co/Parsa/BBB_prediction_classification/commit/1f25e115c75da81d589fa88c2aead630a63dc8e4'